<a href="https://colab.research.google.com/github/mbcruz96/English-to-Spanish-Transformer/blob/main/English_to_Spanish_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Libraries

In [24]:
!pip install datasets
!pip install tokenizers
!pip install tqdm
!pip install tensorboard
!pip install torchmetrics

#Dependencies

In [25]:
import torch
import torch.nn as nn
import math
from typing import Any
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchmetrics
from torchmetrics.text import CharErrorRate, WordErrorRate, BLEUScore
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from tqdm import tqdm

#Mounting Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Model

In [9]:
class TextEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model  # embedding dimensionality
        self.vocab_size = vocab_size    # size of corpus vocabulary
        self.embedding = nn.Embedding(vocab_size, d_model)  # learnable vobabulary embedding

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, sequence_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.sequence_len = sequence_len
        self.dropout = nn.Dropout(dropout)

        # creating a positional encoding matrix of dimension (sequence length, embedding dimensionality)
        pos_enc = torch.zeros(sequence_len, d_model)
        # creating positions vector of dimenstion (sequence length, 1)
        positions = torch.arange(0, sequence_len, dtype=torch.float).unsqueeze(1)
        divisor = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pos_enc[:, ::2] = torch.sin(positions * divisor)
        pos_enc[:, 1::2] = torch.cos(positions * divisor)

        # creating a positional encoding for a batch of embeddings of size (1, sequence length, embedding dimensionality)
        pos_enc = pos_enc.unsqueeze(0)

        # saving positional encoding as non-learnable parameter in same file as model weights
        self.register_buffer('pos_enc', pos_enc)

    def forward(self, x):
        x = x + (self.pos_enc[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

class LayerNormalizationBlock(nn.Module):
    def __init__(self, eps: float = 10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(1, device='cuda')) # Multiplicative parameter
        self.beta = nn.Parameter(torch.zeros(1, device='cuda')) # Addative parameter

    # input: (batch_size, seq_len, emb_dim)
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.layer1 = nn.Linear(d_model, d_ff).to(self.device)  # fc layer of dimension (ff_dim, emb_dim)
        self.layer2 = nn.Linear(d_ff, d_model).to(self.device)  # fc layer of dimension (emb_dim, ff_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input: (batch_size, seq_len, emb_dim, ff_dim) --> (batch_size, seq_len, ff_dim, emb_dim) --> (batch_size, seq_len, emb_dim, ff_dim)
        x = torch.relu(self.layer1(x))
        x = self.dropout(x)
        return self.layer2(x)

class MultiHeadSelfAttentionBlock(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.h = h
        self.dropout = nn.Dropout(dropout)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        assert d_model % h == 0, 'Embedding dimensionality not divisible by number of heads'
        self.d_k = d_model // h

        # learnable weight matrices
        self.w_q = nn.Linear(d_model, d_model).to(self.device)
        self.w_k = nn.Linear(d_model, d_model).to(self.device)
        self.w_v = nn.Linear(d_model, d_model).to(self.device)
        self.w_o = nn.Linear(d_model, d_model).to(self.device)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]

        # input: (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)

        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1**9)
        # (batch, h, seq_len, seq_len)
        attention_scores = torch.softmax(attention_scores, dim=-1)
        if dropout is not None:
            dropout(attention_scores)
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        # input: (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        # input: (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        x, self.attention_scores = MultiHeadSelfAttentionBlock.attention(query, key, value, mask, self.dropout)

        # input: (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k) # contiguous allows tensor shape to be changed in contiguous memory block

        return self.w_o(x)

class ResidualConnection(nn.Module):
    def __init__(self, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalizationBlock()

    def forward(self, skip_layer, prev_layer):
        return skip_layer + self.dropout(prev_layer(self.norm(skip_layer)))

class EncoderBlock(nn.Module):
    def __init__(self, attention_block: MultiHeadSelfAttentionBlock, ff_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.attention_block = attention_block
        self.ff_block = ff_block
        self.dropout = nn.Dropout(dropout)
        self.residual_connections = nn.ModuleList(ResidualConnection(dropout) for _ in range(2))

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.ff_block)
        return x

class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalizationBlock()

    def forward(self, x, src_mask):
        for layer in self.layers:
            x = layer(x, src_mask)
        return self.norm(x)

class DecoderBlock(nn.Module):
    def __init__(self, attention_block: MultiHeadSelfAttentionBlock, cross_attention_block: MultiHeadSelfAttentionBlock, ff_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.attention_block = attention_block
        self.cross_attention_block = cross_attention_block
        self.ff_block = ff_block
        self.dropout = nn.Dropout(dropout)
        self.residual_connections = nn.ModuleList(ResidualConnection(dropout) for _ in range(3))

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.ff_block)
        return x

class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalizationBlock()

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)


class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.projection = nn.Linear(d_model, vocab_size).to(self.device)

    def forward(self, x):
        # input: (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return torch.log_softmax(self.projection(x), dim=-1) # log softmax for numerical stability

class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_pos_enc: PositionalEncoding, tgt_pos_enc: PositionalEncoding, src_embed: TextEmbeddings, tgt_embed: TextEmbeddings, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_pos_enc = src_pos_enc
        self.tgt_pos_enc = tgt_pos_enc
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos_enc(src)
        return self.encoder(src, src_mask)

    def decode(self, tgt, encoder_output, src_mask, tgt_mask,):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos_enc(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def projection(self, x):
        return self.projection_layer(x)

def BuildTransformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, h: int = 8, N: int = 6, d_ff: int = 2048, dropout: float = 0.1) -> Transformer:
    # Create embedding layers
    src_embedding = TextEmbeddings(d_model, src_vocab_size)
    tgt_embedding = TextEmbeddings(d_model, tgt_vocab_size)

    # Create positional encodings
    src_pos_enc = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos_enc = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create encoder blocks
    encoder_blocks = []
    for _ in range(N):
        attention_block = MultiHeadSelfAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create decoder blocks
    decoder_blocks = []
    for _ in range(N):
        attention_block = MultiHeadSelfAttentionBlock(d_model, h, dropout)
        cross_attention_block = MultiHeadSelfAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(attention_block, cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # create encoder and decoder
    encoder = Encoder(encoder_blocks)
    decoder = Decoder(decoder_blocks)
    # create projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    # create transformer
    transformer = Transformer(encoder, decoder, src_pos_enc, tgt_pos_enc, src_embedding, tgt_embedding, projection_layer)
    # Initialize transformer parameters for better learning
    for layer in transformer.parameters():
        if layer.dim() > 1:
            nn.init.xavier_uniform_(layer)

    return transformer

#Dataset

In [29]:
class BilingualDataset(Dataset):
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len) -> None:
        super().__init__()
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.seq_len = seq_len

        self.sos_token = torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_src.token_to_id('[PAD]')], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, index: Any) -> Any:
        src_target_pair = self.ds[index]
        src_txt = src_target_pair['translation'][self.src_lang]
        tgt_txt = src_target_pair['translation'][self.tgt_lang]

        enc_input_tokens = self.tokenizer_src.encode(src_txt).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_txt).ids

        enc_num_pad_tokens = self.seq_len - len(enc_input_tokens) - 2
        dec_num_pad_tokens = self.seq_len - len(dec_input_tokens) - 1

        if enc_num_pad_tokens < 0:
            enc_input_tokens = enc_input_tokens[:self.seq_len - 2]
            enc_num_pad_tokens = 0
        if dec_num_pad_tokens < 0:
            dec_input_tokens = dec_input_tokens[:self.seq_len - 1]
            dec_num_pad_tokens = 0

        # Add SOS and EOS to source text
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_pad_tokens, dtype=torch.int64)
            ],
            dim=0
        )

        # Add SOS to decoder input
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_pad_tokens, dtype=torch.int64)
            ],
            dim=0
        )

        # Add SOS to the label (expected decoder output)
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_pad_tokens, dtype=torch.int64)
            ],
            dim=0
        )

        # Check to ensure the sizes of encoder, decoder, and label texts are of sequence length
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        # data set
        return {
            "encoder_input": encoder_input, # size = seq_len
            "decoder_input": decoder_input, # size = seq_len
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & Causual_Mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len)
            "label": label, # (seq_len)
            "source_text": src_txt,
            "target_text": tgt_txt
        }

# returns a masked matrix with all values above diagonal masked out
def Causual_Mask(size):
    mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
    return mask == 0


#Configurations

In [6]:
def Get_Config():
    return{
        "batch_size": 8,
        "num_epochs": 20,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "lang_src": "en",
        "lang_tgt": "es",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "drive_path": "/content/drive/MyDrive/Colab/TranslationTransformer/",
        "preload": "latest",
        "tokenizer_filename": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

def Get_Weights_File_Path(config, epoch: str):
    drive_path = config['drive_path']
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('.') / drive_path / model_folder / model_filename)

def Get_Tokenizer_File_Path(config, lang: str):
    drive_path = config['drive_path']
    tokenizer_filename = Path(config['tokenizer_filename'].format(lang))
    return str(Path('.') / drive_path / tokenizer_filename)

def Get_Model_File_Path(config):
    drive_path = config['drive_path']
    model_folder = config['model_folder']
    return str(Path('.') / drive_path / model_folder)

# Find the latest weights file in the weights folder
def Latest_Weights_File_Path(config):
    model_folder = f"{config['model_folder']}"
    model_filename = f"{config['model_basename']}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])

#Inference

In [31]:
def Greedy_Decode(model, device, encoder_input, encoder_mask, src_tokenizer, tgt_tokenizer, max_len):
    sos_idx = tgt_tokenizer.token_to_id('[SOS]')
    eos_idx = tgt_tokenizer.token_to_id('[EOS]')

    # Precompute encoder output and reuse it for every token we get from the decoder
    encoder_output = model.encode(encoder_input, encoder_mask)
    # Initialize the decoder output with the SOS token
    decoder_input = torch.empty(1,1).fill_(sos_idx).type_as(encoder_input).to(device)

    while True:
        # Break if decoder is max sequence length
        if decoder_input.size(1) == max_len:
            break

        # build mask for the current decoder input
        decoder_mask = Causual_Mask(decoder_input.size(1)).type_as(encoder_mask).to(device)

        # calculate the decoder output
        output = model.decode(decoder_input, encoder_output, encoder_mask, decoder_mask)

        # get the next token
        prob = model.projection(output[:, -1]) # project the output for the last token in the sequence
        # select the token with the next highest probability
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1,1).type_as(encoder_input).fill_(next_word.item()).to(device)], dim=1)

        # break if next word is eos token
        if next_word == eos_idx:
            break

    # return generated sequence without batch dimension
    return decoder_input.squeeze(0)

def Run_Validation(model, device, val_ds, src_tokenizer, tgt_tokenizer, max_len, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    console_width = 80 # size of control window

    with torch.no_grad():
        for batch in val_ds:
            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            # ensure validation batch is 1
            assert encoder_input.size(0) == 1, 'Batch size must be 1 for validation'

            # generate the output tokens
            model_out = Greedy_Decode(model, device, encoder_input, encoder_mask, src_tokenizer, tgt_tokenizer, max_len)

            # get actual model output
            source_text = batch['source_text'][0]
            target_text = batch['target_text'][0]
            model_out_text = tgt_tokenizer.decode(model_out.detach().cpu().numpy())

             # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break

            if writer:
                # Evaluate the character error rate
                # Compute the char error rate
                metric = torchmetrics.CharErrorRate()
                cer = metric(model_out_text, target_text)
                writer.add_scalar('validation cer', cer, global_step)
                writer.flush()

                # Compute the word error rate
                metric = torchmetrics.WordErrorRate()
                wer = metric(model_out_text, target_text)
                writer.add_scalar('validation wer', wer, global_step)
                writer.flush()

                # Compute the BLEU metric
                metric = torchmetrics.BLEUScore()
                bleu = metric(model_out_text, target_text)
                writer.add_scalar('validation BLEU', bleu, global_step)
                writer.flush()

#Train

In [32]:
def Get_All_Sentances(ds, lang):
    for item in ds:
        yield item['translation'][lang]

def Get_or_Build_Tokenizer(config, ds, lang):
    tokenizer_path = Get_Tokenizer_File_Path(config, lang)
    if not Path(tokenizer_path).exists():
        print('Tokenizer not found, building...')
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=['[UNK]', '[PAD]', '[SOS]', '[EOS]'], min_frequency=2)
        tokenizer.train_from_iterator(Get_All_Sentances(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        print('Tokenizer found, loading...')
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

def Get_Dataset(config):
    ds_raw = load_dataset('opus_books', f'{config["lang_src"]}-{config["lang_tgt"]}', split='train')

    # Built tokenizer
    tokenizer_src = Get_or_Build_Tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = Get_or_Build_Tokenizer(config, ds_raw, config['lang_tgt'])

    # Split train and validation sets
    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) -  train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])


    max_len_src = 0
    max_len_tgt = 0

    for item in ds_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentance: {max_len_src}')
    print(f'Max length of target sentance: {max_len_tgt}')

    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

def Get_Model(config, src_vocab_size, tgt_vocab_size):
    model = BuildTransformer(src_vocab_size, tgt_vocab_size, config['seq_len'], config['seq_len'], config['d_model'])
    return model

def Train_model(config):
    # setting device to train on
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device {device}')

    # Create file for parameters to be saved to
    model_folder = Get_Model_File_Path(config)
    Path(model_folder).mkdir(parents=True, exist_ok=True)

    # Initialize dataloaders and tokenizers
    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = Get_Dataset(config)

    # Initialize model
    model = Get_Model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    # Create a tensorboard for loss visualization
    writer = SummaryWriter(config['experiment_name'])

    # Initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

    # Initial epoch start and global stop of this training cycle
    initial_epoch = 0
    global_step = 0

    # preload previous model parameters if they exist
    if config['preload'] == "latest":
        model_filename = Latest_Weights_File_Path(config)
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)  # loading previous model state
        initial_epoch = state['epoch'] + 1
        model.load_state_dict(state['model_state_dict'])
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
        print(f'Starting training at epoch {initial_epoch}')
    else:
      model_filename = Get_Model_File_Path(config, config['preload'])
      print(f'No model found to preload')

    # Initializing loss function
    # Ignores any padded tokens in loss computation
    # Smoothes result by distributing .1 of argmax value to all other labels
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    # Training loop
    for epoch in range(initial_epoch, config['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        # Initialize batch iterator using tqdm for a progression bar visualization
        batch_iterator = tqdm(train_dataloader, desc=f'Processing epoch {epoch:02d}')
        for batch in batch_iterator:
            if batch == None:
                continue
            encoder_input = batch['encoder_input'].to(device)   # input: (batch, seq_len)
            decoder_input = batch['decoder_input'].to(device)   # input: (batch, seq_len)
            encoder_mask = batch['encoder_mask'].to(device)     # (batch, 1, 1, seq_len) to mask padded tokens
            decoder_mask = batch['decoder_mask'].to(device)     # (batch, 1, seq_len, seq_len) to mask padded tokens and future words

            # run tensors through model
            encoder_output = model.encode(encoder_input, encoder_mask)  # (batch, seq_len, d_model)
            decoder_output = model.decode(decoder_input, encoder_output, encoder_mask, decoder_mask)    # (batch, seq_len, d_model)
            projection_output = model.projection(decoder_output)    # (batch, seq_len, tgt_vocab_size)
            # Get label from batch
            label = batch['label'].to(device)

            # input (batch, seq_len, tgt_vocab_size) --> (batch * seq_len, tgt_vocab_size)
            loss = loss_fn(projection_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))

            # update the progress bar with the loss
            batch_iterator.set_postfix({f'loss': f'{loss.item():6.3f}'})

            # log the loss in tensorboard
            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            # Backpropogate the loss
            loss.backward()

            # update the weights
            optimizer.step()
            optimizer.zero_grad()

            global_step += 1

        # Save the model at the end of every epoch
        model_filename = Get_Weights_File_Path(config, f'{epoch:02d}')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

        # Run validation at the end of each epoch
        Run_Validation(model, device, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], lambda msg: batch_iterator.write(msg), global_step, writer)

#Main

In [33]:
config = Get_Config()
Train_model(config)

Using device cuda
Tokenizer found, loading...
Tokenizer found, loading...
Max length of source sentance: 767
Max length of target sentance: 782


Processing epoch 00: 100%|██████████| 10516/10516 [26:30<00:00,  6.61it/s, loss=4.629]


--------------------------------------------------------------------------------
    SOURCE: How could you have consoled her!--I cannot express my own abhorrence of myself.
    TARGET: ¡Cómo habría perdurado en tus recuerdos! ¡Y mi madre, también!
 PREDICTED: - Pero , que la señora , que la que la que la que la que la , que la , que la , que la , que la , que la , que la , que la , que , que , que , que , que no , que , , , , que , , , , que , , , , , , , , , , , , , que , , , , , , , , , , de


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


--------------------------------------------------------------------------------
    SOURCE: And the Eaglet bent down its head to hide a smile: some of the other birds tittered audibly.
    TARGET: Y el Aguilucho bajó la cabeza para ocultar una sonrisa; algunos de los otros pájaros rieron sin disimulo.
 PREDICTED: Y , que la , que la , que la , , que la , , , , , ,
--------------------------------------------------------------------------------


...

--------------------------------------------------------------------------------
    SOURCE: The prisoner turned with the reckless air of a man who abandons himself to his destiny. "Be it so," said he.
    TARGET: ¡Lo reconozco por las fotografías! El preso se volvió con el aire indiferente de quien se abandona en manos del destino. ––De acuerdo ––dijo––.
 PREDICTED: Sea dijo el destino , que el destino ; el destino , que el destino , que se volvió a su destino ; el destino ; el destino .
--------------------------------------------------------------------------------
    SOURCE: "Then you have been in Paraguay?" asked Candide.
    TARGET: ¿Con que tu ya has estado en el Paraguay? le dixo Candido.
 PREDICTED: ¿ Y en el caso ? – preguntó el momento ? - preguntó Candido .
--------------------------------------------------------------------------------


Processing epoch 18: 100%|██████████| 10516/10516 [26:29<00:00,  6.62it/s, loss=3.899]


--------------------------------------------------------------------------------
    SOURCE: And is Philip the gardener really still living?
    TARGET: ¿Y la casa? ¿Sigue como antes?
 PREDICTED: Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ¿ Y ahora ?
--------------------------------------------------------------------------------
    SOURCE: D’Artagnan fought three times with Rochefort, and wounded him three times.
    TARGET: D'Artagnan se batió tres veces con Rochefort y lo hirió tres veces.
 PREDICTED: D ' Artagnan tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces tres veces t

Processing epoch 19: 100%|██████████| 10516/10516 [26:30<00:00,  6.61it/s, loss=3.646]


--------------------------------------------------------------------------------
    SOURCE: The information about Mr. John's death and the manner of it came too suddenly: it brought on a stroke.
    TARGET: Las pérdidas de dinero y el temor a la pobreza la han empeorado. Y la brusca noticia del suicidio del señorito le produjo un ataque.
 PREDICTED: El señor John apareció una información , y se trata de pronto , y la muerte de pronto , y muy poco de pronto como un poco de repente .
--------------------------------------------------------------------------------
    SOURCE: I have a horror of fallen women.
    TARGET: Aborrezco a las mujeres perdidas.
 PREDICTED: ¿ Y las mujeres , y las mujeres , y las mujeres .
--------------------------------------------------------------------------------
